In [1]:
from spectate import Sentinel, Spectator, WatchedType

In [2]:
Empty = Sentinel('Empty')

class DictSpectator(Spectator):
    
    def __init__(self, inst):
        super(DictSpectator, self).__init__(inst)
        self._change = {'old': None, 'new': None}

    def instance_will_call(self, name, args, kwargs):
        self._handlers[name](self, *args, **kwargs)
    
    def _write_change(self, k, v):
        if self._change['old'] is not None:
            o = self._change['old']
        else:
            o = {}
            self._change['old'] = o
        if self._change['new'] is not None:
            n = self._change['new']
        else:
            n = {}
            self._change['new'] = n
        o[k] = self.inst.get(k, Empty)
        n[k] = v

    def _will_setitem(self, k, v):
        if k not in self.inst or v != self.inst[k]:
            self._write_change(k, v)
            
    def _will_setdefault(self, k, v):
        if k not in self.inst:
            self._write_change(k, v)
    
    def _will_update(self, d):
        for k, v in d.items():
            self._will_setitem(k, v)
    
    def _will_pop(self, k):
        if k in self.inst:
            self._write_change(k, Empty)
            
    def _will_popitem(self):
        k = self.inst.keys()[0]
        self._write_change(k, Empty)
    
    _handlers = {'__setitem__': _will_setitem, 'setdefault': _will_setdefault,
        'update': _will_update, 'pop': _will_pop, 'popitem': _will_popitem}
         
    def instance_post_call(self, name, returned):
        o, n = self._change['old'], self._change['new']
        if o and n:
            print('%s : %r -> %r' % (name, o, n))
            self._change['old'] = None
            self._change['new'] = None

In [3]:
watched_methods = ('__setitem__', 'pop', 'update', 'setdefault', 'popitem')
WatchedDict = WatchedType('WatchedDict', dict, DictSpectator, watched_methods)

In [4]:
wd = WatchedDict({'a': 1, 'b':2})
sp = wd.instance_spectator

msg = "spectator is linked to instance: %s"
print(msg % (sp.inst is wd))

spectator is linked to instance: True


In [5]:
# tests
wd['a'] = 2
wd.pop('a')
wd.popitem()
wd.setdefault('c', 3)
wd.update({'a':1, 'b':2})
# won't notify
wd['a'] = 1

__setitem__ : {'a': 1} -> {'a': 2}
pop : {'a': 2} -> {'a': Empty}
popitem : {'b': 2} -> {'b': Empty}
setdefault : {'c': Empty} -> {'c': 3}
update : {'a': Empty, 'b': Empty} -> {'a': 1, 'b': 2}
